### Import and Setup

In [4]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import collections
import yaml
from matplotlib import pyplot as plt
from nltk.util import ngrams

annotypes = ['Participants', 'Intervention', 'Outcome']

### Model

### Autoslog for Syntactic Patterns

In [5]:
fnames = [annotype + '.slist' for annotype in annotypes]

In [6]:
#shell script to run aslog system, run on grid within folder

'''
cd /nlp/data/romap/autoslog/sundance-v5.1/bin
./aslog -r /nlp/data/romap/autoslog/inputs/part_relevant.txt -l /nlp/data/romap/autoslog/inputs/part.slist -u 2 -o /nlp/data/romap/autoslog/inputs/part_all/
./aslog -2 /nlp/data/romap/autoslog/inputs/part_all/part.slist.caseframes_1 -r /nlp/data/romap/autoslog/inputs/part_relevant.txt -l /nlp/data/romap/autoslog/inputs/part.slist -u 2 -o /nlp/data/romap/autoslog/inputs/part_all/
./aslog -r /nlp/data/romap/autoslog/inputs/out_relevant.txt -l /nlp/data/romap/autoslog/inputs/out.slist -u 2 -o /nlp/data/romap/autoslog/inputs/out_all/
./aslog -2 /nlp/data/romap/autoslog/inputs/out_all/out.slist.caseframes_1 -r /nlp/data/romap/autoslog/inputs/out_relevant.txt -l /nlp/data/romap/autoslog/inputs/out.slist -u 2 -o /nlp/data/romap/autoslog/inputs/out_all/
./aslog -r /nlp/data/romap/autoslog/inputs/int_relevant.txt -l /nlp/data/romap/autoslog/inputs/int.slist -u 2 -o /nlp/data/romap/autoslog/inputs/int_all/
./aslog -2 /nlp/data/romap/autoslog/inputs/int_all/int.slist.caseframes_1 -r /nlp/data/romap/autoslog/inputs/int_relevant.txt -l /nlp/data/romap/autoslog/inputs/int.slist -u 2 -o /nlp/data/romap/autoslog/inputs/int_all/romap@nlpgrid:/nlp/data/romap/autoslog/inputs>
'''

'\ncd /nlp/data/romap/autoslog/sundance-v5.1/bin\n./aslog -r /nlp/data/romap/autoslog/inputs/part_relevant.txt -l /nlp/data/romap/autoslog/inputs/part.slist -u 2 -o /nlp/data/romap/autoslog/inputs/part_all/\n./aslog -2 /nlp/data/romap/autoslog/inputs/part_all/part.slist.caseframes_1 -r /nlp/data/romap/autoslog/inputs/part_relevant.txt -l /nlp/data/romap/autoslog/inputs/part.slist -u 2 -o /nlp/data/romap/autoslog/inputs/part_all/\n./aslog -r /nlp/data/romap/autoslog/inputs/out_relevant.txt -l /nlp/data/romap/autoslog/inputs/out.slist -u 2 -o /nlp/data/romap/autoslog/inputs/out_all/\n./aslog -2 /nlp/data/romap/autoslog/inputs/out_all/out.slist.caseframes_1 -r /nlp/data/romap/autoslog/inputs/out_relevant.txt -l /nlp/data/romap/autoslog/inputs/out.slist -u 2 -o /nlp/data/romap/autoslog/inputs/out_all/\n./aslog -r /nlp/data/romap/autoslog/inputs/int_relevant.txt -l /nlp/data/romap/autoslog/inputs/int.slist -u 2 -o /nlp/data/romap/autoslog/inputs/int_all/\n./aslog -2 /nlp/data/romap/autoslog

In [7]:
#include these in config for LSTM
#change build_data in LSTM for pattern indicators

pnames = [annotype + '_patterns.yaml' for annotype in annotypes]

### LSTM-CRF Model

In [8]:
#shell to run variants of same model and evaluate on grid

'''source /home1/r/romap/crf/crf_task/bin/activate
python /nlp/data/romap/naacl-pattern/lstm/joint/model_1/build_data.py
python /nlp/data/romap/naacl-pattern/lstm/joint/model_1/train.py
python /nlp/data/romap/naacl-pattern/lstm/joint/model_1/evaluate.py
'''

'source /home1/r/romap/crf/crf_task/bin/activate\npython /nlp/data/romap/naacl-pattern/lstm/joint/model_1/build_data.py\npython /nlp/data/romap/naacl-pattern/lstm/joint/model_1/train.py\npython /nlp/data/romap/naacl-pattern/lstm/joint/model_1/evaluate.py\n'

### Simple (test) LSTM

In [9]:
def RNN(x, weights, biases):
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_input, axis=1)
    rnn_cell = rnn.BasicLSTMCell(n_hidden)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    final = tf.matmul(output[-1], (weights['out'] + biases['out']))
    print final
    return final
    
def create_dataset(words):
    #build lookup dictionary and reverse dictionary
    count = collections.Counter(words)
    dictionary, reverse_dictionary = {}, {}
    for word in count:
        reverse_dictionary[(len(dictionary))] = word
        dictionary[word] = len(dictionary)
    return dictionary, reverse_dictionary
        
create_dataset(['hi', 'Brown', 'is', 'great'])
    
def get_words():
    s = 'had a general council to consider what measures they could take to outwit their common enemy , the cat . some said this , and some said that but at last a young mouse got'
    return s.split(' ')
    
def test():
    words = get_words()
    dictionary, reverse_dictionary = create_dataset(words)
    n_input, n_hidden, vocab_size, offset = 3, 512, len(dictionary.keys()), 0
    weights = {'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
    biases = {'out': tf.Variable(tf.random_normal([vocab_size]))}
                      
        
    inputs = [dictionary[str(words[i])] for i in range(offset, (offset + n_input))]
    outputs = np.zeros([vocab_size], dtype=float)
    outputs[dictionary[str(words[offset + n_input])]] = 1.0
    session = tf.Session()
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)


    _, acc, loss, onehot_pred = session.run(
        [optimizer, accuracy, cost, pred], 
        feed_dict={x: inputs, 
        y: outputs})
                      
    pred = RNN(x, weights, biases)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
                      
                      
                      
test()

NameError: global name 'learning_rate' is not defined